In [1]:
using CSV, DataFrames, GLM, RegressionTables, FixedEffectModels

In [2]:
walk15_gini = CSV.read("../output/budapest/20250428/walk15_stop_gini.csv", DataFrame)
multimodal_gini = CSV.read("../output/budapest/20250428/multimodal_stop_gini.csv", DataFrame)
indicators = CSV.read("../output/budapest/20250428/merged.csv", DataFrame)
;

In [3]:
gini = innerjoin(walk15_gini, multimodal_gini, on = :stop_id)
data = innerjoin(indicators, gini, on = :stop_id)
;

In [ ]:
CSV.write("../output/budapest/20250428/data_merged_in_julia.csv", data)
;

In [ ]:
data[!, :gini_change] = data[!, :multimodal_gini] .- data[!, :walk15_gini]
;

In [ ]:
first(data, 3)

Row,stop_id,area,ellipticity,number_of_accessible_stops,concave_area,distance_betweenness,distance_from_landuse_centroid,cultural_institutions_multimodal,drugstores_multimodal,groceries_multimodal,healthcare_multimodal,parks_multimodal,religious_organizations_multimodal,restaurants_multimodal,schools_multimodal,services_multimodal,cultural_institutions_walk15,drugstores_walk15,groceries_walk15,healthcare_walk15,parks_walk15,religious_organizations_walk15,restaurants_walk15,schools_walk15,services_walk15,walk_area,area_difference,stop_name,stop_lat,stop_lon,cluster,eigenvector_centrality,degree_centrality,closeness_centrality,betweenness_centrality,walk15_gini,multimodal_gini,gini_change
,String15,Float64,Float64?,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,String,Float64,Float64,Int64,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64
1,007877,3.094,0.9241,32,0.513,7.154,8.781,2,5,6,10,22,6,17,12,9,2,2,3,2,11,3,2,12,2,2.081,1.013,Bíróság,47.5195,18.9781,2120,3.23147e-7,0.000983768,0.0540533,0.00686604,0.139,0.1473,0.0083
2,007878,3.874,0.8772,46,0.77,7.15,8.78,2,6,11,14,31,8,26,24,15,2,2,3,2,11,3,2,12,2,2.099,1.775,Bíróság,47.5194,18.978,2120,3.23147e-7,0.000983768,0.0540533,0.00686604,0.139,0.1484,0.0094
3,007879,2.639,0.425,16,1.044,9.057,8.701,5,5,14,3,35,2,50,10,24,3,2,10,2,13,1,46,4,15,2.298,0.341,"Római úti lakótelep, Varsa utca",47.5709,19.0585,1752,6.26071e-9,0.000491884,0.049765,0.0,0.1582,0.171,0.0128


In [ ]:
# m0 = reg(data, @formula(gini_change ~ area + distance_betweenness + closeness_centrality + ellipticity))
m2 = reg(data, @formula(multimodal_gini ~ walk15_gini + area + distance_betweenness + closeness_centrality + ellipticity))

                                        FixedEffectModel                                        
Number of obs:                             4425  Converged:                                 true
dof (model):                                  5  dof (residuals):                           4418
R²:                                       0.204  R² adjusted:                              0.203
F-statistic:                            226.782  P-value:                                  0.000
                          Estimate   Std. Error     t-stat  Pr(>|t|)     Lower 95%     Upper 95%
────────────────────────────────────────────────────────────────────────────────────────────────
walk15_gini            0.286285     0.0090286    31.7086      <1e-99   0.268584      0.303985
area                  -5.56795e-5   0.000143451  -0.388143    0.6979  -0.000336915   0.000225556
distance_betweenness   0.000497824  0.000102409   4.86114     <1e-05   0.000297051   0.000698596
closeness_centrality   0.385421  

In [ ]:
regtable([m2]...; render=AsciiTable(), file="m1.txt")


--------------------------------------
                       multimodal_gini
--------------------------------------
(Intercept)                   0.099***
                               (0.003)
walk15_gini                   0.286***
                               (0.009)
area                            -0.000
                               (0.000)
distance_betweenness          0.000***
                               (0.000)
closeness_centrality          0.385***
                               (0.032)
ellipticity                   -0.004**
                               (0.001)
--------------------------------------
N                                4,425
R2                               0.204
--------------------------------------

